# Optimizing NBA Roster



Introduction:
This report was designed to optimize an NBA roster based on current 2018-2019 season statistics. Improvements in data have led to a greater place in sports for statistics to add what coaches and experts are unable to see. I

In [119]:
using JuMP, Gurobi, Cbc

data = readcsv("players_clean.csv")
#m = Model(solver=GurobiSolver(OutputFlag=0));
m = Model(solver=CbcSolver())
lambda = 0.5 

#make data table without catagory headers
row = length(data[1,:])
col = length(data[:,3])
data = data[2:col, 2:row]

#zip players to their respective statistics
player_names = data[:,2]
player_pts = Dict(zip(player_names, data[:,5]))
player_trb = Dict(zip(player_names, data[:,6]))
player_ast = Dict(zip(player_names, data[:,7]))
player_stl = Dict(zip(player_names, data[:,8]))
player_sal = Dict(zip(player_names, data[:,9]))

#a bit array is pushed to the positions matrix depending on position for every player
positions = []
for i in data[:,4]
    if i == "PG"
        push!(positions, [1  0  0  0  0]) 
    end
    if i == "SG"
        push!(positions, [0  1  0  0  0])
    end
    if i == "SF"
        push!(positions, [0  0  1  0  0])
    end
    if i == "PF"
        push!(positions, [0  0  0  1  0])
    end
    if i == "C"
        push!(positions, [0  0  0  0  1])
    end
end

player_pos = Dict(zip(player_names, positions))

#may only have 2 of each position on each 10 man roster
position_max = [2  2  2  2  2]

#sum stats 
#@variable(m, x[player_names])
@variable(m, x[player_names], Bin)
@expression(m, points, sum([player_pts[p] for p in player_names]))
@expression(m, rebounds, sum([player_trb[p] for p in player_names]))
@expression(m, assists, sum([player_ast[p] for p in player_names]))
@expression(m, steals, sum([player_stl[p] for p in player_names]))
@expression(m, salaries, sum([player_sal[p]*x[p] for p in player_names]))
@expression(m, stats, points + rebounds + assists + steals)
@expression(m, total_pos, sum([player_pos[p] for p in player_names]))
@constraint(m, total_pos .== position_max)
@constraint(m, sum(x) == 10)
@objective(m, Min, -stats - lambda*salaries)

#solve(m)
solve(m)

:Infeasible